# Sudoku Solver

This program takes text files of 9x9 grids of numbers as input, where a 0 represents a blank square. See https://projecteuler.net/problem=96 for sample puzzles.

**Example input:** C:\\Users\\Ariel\\Python Workspace\\Sudoku1.txt

Currently, this solver consists of three functions:

**OnePossible():**

This function checks each section (rows, columns and boxes) to see if there are any squares that only have one possible number they could be. For example, if the row containing the square had the numbers 1 through 5, and the column containing the square had the numbers 6 through 8, this function would find that the only possible choice is 9. It would then make that move and update the puzzle as well as the options for each empty square.

**OnlyPossible():**

This function checks if a number can only be in one square within a section (row, column or box). For example, say there are 3 spaces left within a box. The possibilities for the first are 1, 2 and 3, and the possibilities for
both the second and third squares are 1 and 2. This algorithm would find that 3 is only possible in the first square, and perform that move.

**NakedGroups():**

This function checks if there are any groups of _n_ squares in a section (row, column, or box) that have the same _n_ options available. This is called a naked group. Then it removes these _n_ options from every other square in the section. For example, if the only options of two squares in a row are 1 and 2, we know that 1 and 2 must go in these squares and not any of the other squares in the row.

In [10]:
#FUNCTION: GenerateOptions(r, c)
#Returns a list of possible numbers for an empty square, given a pair of coordinates
#If a number is already printed, returns an empty list []
def GenerateOptions(r, c):
    
    #Generate full list of options 1-9:
    o_list = [x for x in range(1,10)]
    
    #If square is not empty, remove all options:
    if puzzle[r][c] != 0:
        o_list = []
    
    #Continue to refine options if the square is empty:
    else:
        
        #Check row, remove all numbers from the options that exist in that row:
        for y in range(0,9):
            if puzzle[r][y] in o_list:
                o_list.remove(puzzle[r][y])


        #Check col, remove all numbers from the options that exist in that col:
        for x in range(0,9):
            if puzzle[x][c] in o_list:
                o_list.remove(puzzle[x][c])

        #Check box, remove all numbers from the options that exist in that box:
        #Boxes numbered 1-9 left to right, top to bottom

        #Box 1
        if r < 3 and c < 3:
            for x in range(0,3):
                for y in range(0,3):
                    if puzzle[x][y] in o_list:
                        o_list.remove(puzzle[x][y])

        #Box 2
        elif r < 3 and 2 < c < 6:
            for x in range(0,3):
                for y in range(3,6):
                    if puzzle[x][y] in o_list:
                        o_list.remove(puzzle[x][y])

        #Box 3
        elif r < 3 and c > 5:
            for x in range(0,3):
                for y in range(6,9):
                    if puzzle[x][y] in o_list:
                        o_list.remove(puzzle[x][y])

        #Box 4
        elif 2 < r < 6 and c < 3:
            for x in range(3,6):
                for y in range(0,3):
                    if puzzle[x][y] in o_list:
                        o_list.remove(puzzle[x][y])

        #Box 5
        elif 2 < r < 6 and 2 < c < 6:
            for x in range(3,6):
                for y in range(3,6):
                    if puzzle[x][y] in o_list:
                        o_list.remove(puzzle[x][y])

        #Box 6
        elif 2 < r < 6 and c > 5:
            for x in range(3,6):
                for y in range(6,9):
                    if puzzle[x][y] in o_list:
                        o_list.remove(puzzle[x][y])

        #Box 7
        elif r > 5 and c < 3:
            for x in range(6,9):
                for y in range(0,3):
                    if puzzle[x][y] in o_list:
                        o_list.remove(puzzle[x][y])

        #Box 8
        elif r > 5 and 2 < c < 6:
            for x in range(6,9):
                for y in range(3,6):
                    if puzzle[x][y] in o_list:
                        o_list.remove(puzzle[x][y])

        #Box 9
        elif r > 5 and c > 5:
            for x in range(6,9):
                for y in range(6,9):
                    if puzzle[x][y] in o_list:
                        o_list.remove(puzzle[x][y])

        else:
            print("ERROR")
        
                              
    return o_list
    
    
#FUNCTION: MakeMove(r, c, v)
#Updates the board at row r and col c with value v
#Also updates all affected options lists  
def MakeMove(r, c, v):
    
    #Making the move:
    puzzle[r][c] = v
                              
    #Updating the affected options:
    
    #Affected row:
    for y in range(0,9):
        if v in options[r][y]:
            options[r][y].remove(v)
        
    #Affected col:
    for x in range(0,9):
        if v in options[x][c]:
            options[x][c].remove(v)
        
    #Affected box:
    #Boxes numbered 1-9 left to right, top to bottom
    
    #Box 1
    if r < 3 and c < 3:
        for x in range(0,3):
            for y in range(0,3):
                if v in options[x][y]:
                    options[x][y].remove(v)
        
    #Box 2
    elif r < 3 and 2 < c < 6:
        for x in range(0,3):
            for y in range(3,6):
                if v in options[x][y]:
                    options[x][y].remove(v)
    
    #Box 3
    elif r < 3 and c > 5:
        for x in range(0,3):
            for y in range(6,9):
                if v in options[x][y]:
                    options[x][y].remove(v)
        
    #Box 4
    elif 2 < r < 6 and c < 3:
        for x in range(3,6):
            for y in range(0,3):
                if v in options[x][y]:
                    options[x][y].remove(v)
    
    #Box 5
    elif 2 < r < 6 and 2 < c < 6:
        for x in range(3,6):
            for y in range(3,6):
                if v in options[x][y]:
                    options[x][y].remove(v)
    
    #Box 6
    elif 2 < r < 6 and c > 5:
        for x in range(3,6):
            for y in range(6,9):
                if v in options[x][y]:
                    options[x][y].remove(v)
    
    #Box 7
    elif r > 5 and c < 3:
        for x in range(6,9):
            for y in range(0,3):
                if v in options[x][y]:
                    options[x][y].remove(v)
    
    #Box 8
    elif r > 5 and 2 < c < 6:
        for x in range(6,9):
            for y in range(3,6):
                if v in options[x][y]:
                    options[x][y].remove(v)
    
    #Box 9
    elif r > 5 and c > 5:
        for x in range(6,9):
            for y in range(6,9):
                if v in options[x][y]:
                    options[x][y].remove(v)
    
    else:
        print("ERROR")
  
    

#FUNCTION: OnePossible()
#Checks all squares to see if they only have one option, then makes that move
#Returns the count of moves made during the function call
def OnePossible():
    
    move_count = 0
    
    #For each square:
    for x in range(0,9):
        for y in range(0,9):
            
            #If there is only one option, make that move and add 1 to the move count:
            if len(options[x][y]) == 1:
                
                MakeMove(x, y, options[x][y][0])
                move_count += 1
    
    return move_count



#FUNCTION: OnlyPossible()
#Checks all squares to see if a number can only be in one square within a section (row, col, or box), then makes that move
#Returns the count of moves made during the function call
def OnlyPossible():

    move_count = 0
    
    #For every square:
    for x in range(0,9):
        for y in range(0,9):
            
            #If this square is empty:
            if puzzle[x][y] == 0:     
            
                #For every option for that square:
                for v in options[x][y]:

                    make_move = True

                    #Check if this option is available to any other square in the row, (except the one in question):
                    for col in range(0,9):
                        if v in options[x][col] and col != y:
                            make_move = False
                            
                    #Check if this option is available to any other square in the col:
                    for row in range(0,9):
                        if v in options[row][y] and row != y:
                            make_move = False
                            
                    #Check if this option is available to any other square in the box:
                    #Box 1
                    if x < 3 and y < 3:
                        for row in range(0,3):
                            for col in range(0,3):
                                if v in options[row][col]:
                                    if row != x and col !=y:
                                        make_move = False

                    #Box 2
                    elif x < 3 and 2 < y < 6:
                        for row in range(0,3):
                            for col in range(3,6):
                                if v in options[row][col]:
                                    if row != x and col !=y:
                                        make_move = False

                    #Box 3
                    elif x < 3 and y > 5:
                        for row in range(0,3):
                            for col in range(6,9):
                                if v in options[row][col]:
                                    if row != x and col !=y:
                                        make_move = False

                    #Box 4
                    elif 2 < x < 6 and y < 3:
                        for row in range(3,6):
                            for col in range(0,3):
                                if v in options[row][col]:
                                    if row != x and col !=y:
                                        make_move = False

                    #Box 5
                    elif 2 < x < 6 and 2 < y < 6:
                        for row in range(3,6):
                            for col in range(3,6):
                                if v in options[row][col]:
                                    if row != x and col !=y:
                                        make_move = False

                    #Box 6
                    elif 2 < x < 6 and y > 5:
                        for row in range(3,6):
                            for col in range(6,9):
                                if v in options[row][col]:
                                    if row != x and col !=y:
                                        make_move = False

                    #Box 7
                    elif x > 5 and y < 3:
                        for row in range(6,9):
                            for col in range(0,3):
                                if v in options[row][col]:
                                    if row != x and col !=y:
                                        make_move = False

                    #Box 8
                    elif x > 5 and 2 < y < 6:
                        for row in range(6,9):
                            for col in range(3,6):
                                if v in options[row][col]:
                                    if row != x and col !=y:
                                        make_move = False

                    #Box 9
                    elif x > 5 and y > 5:
                        for row in range(6,9):
                            for col in range(6,9):
                                if v in options[row][col]:
                                    if row != x and col !=y:
                                        make_move = False

                    else:
                        print("ERROR")
                    
                    
                    #If all checks have been passed, make move and add to move count:
                    if make_move == True:
                        MakeMove(x, y, v)
                        move_count += 1

    return move_count

#FUNCTION: NakedGroups()
#Checks for pairs in a section (row, column or box) that contain the same two numbers as options, ex. 1,2 and 1,2.
#1 and 2 can then be removed from all other options lists in that section, since 1 and 2 must go in those squares.
def NakedGroups():
    
    #For each row:
    for r in range(0,9):
        
        for c in range(0,9):
            
            #List of squares that have equal options, and a count of how many squares have equal options:
            #(Resets variables for the next loop)
            equal_list = []
            equal_count = 1
            
            #Check if square's options are equal to any other square in that row (if it appears after c):
            for c2 in range(c+1,9):
                if c2 != c and options[r][c] == options[r][c2]:
                    
                    #Add [r,c] to the list if it is the first time an equal has been found:
                    if equal_count == 1:
                        equal_list.append([r,c])
                        
                    #Add [r,c2] if equal and add to the count:
                    equal_list.append([r,c2])
                    equal_count += 1
                    
            #Equal_list is complete for square [r,c] for row r
            #If an equal was found, check that the number of equals found is equal to the number of options for [r,c]
            if equal_count == len(options[r][c]) and equal_count > 1:
                
                #A naked group has been found, remove every option in options[r][c] from every square in the row:
                for option in options[r][c]:
                    for c3 in range(0,9):
                        
                        #Checks if the square checked is not in the naked group:
                        in_group = False
                        for ng_square in equal_list:
                            if c3 == ng_square[1]:
                                in_group = True
                                
                        #If the square is not in the group, remove the options:
                        if in_group == False:  
                            if option in options[r][c3]:
                                options[r][c3].remove(option)
            
                    
    
    #For each col:
    for c in range(0,9):
        
        for r in range(0,9):
            
            #List of squares that have equal options, and a count of how many squares have equal options:
            #(Resets variables for the next loop)
            equal_list = []
            equal_count = 1
            
            #Check if square's options are equal to any other square in that col (if it appears after r):
            for r2 in range(r+1,9):
                if r2 != r and options[r][c] == options[r2][c]:
                    
                    #Add [r,c] to the list if it is the first time an equal has been found:
                    if equal_count == 1:
                        equal_list.append([r,c])
                        
                    #Add [r2,c] if equal and add to the count:
                    equal_list.append([r2,c])
                    equal_count += 1
                    
            #Equal_list is complete for square [r,c] for col c
            #If an equal was found, check that the number of equals found is equal to the number of options for [r,c]
            if equal_count == len(options[r][c]) and equal_count > 1:
                
                #A naked group has been found, remove every option in options[r][c] from every square in the col:
                for option in options[r][c]:
                    for r3 in range(0,9):
                        
                        #Checks if the square checked is not in the naked group:
                        in_group = False
                        for ng_square in equal_list:
                            if r3 == ng_square[0]:
                                in_group = True
                                
                        #If the square is not in the group, remove the options:
                        if in_group == False:  
                            if option in options[r3][c]:
                                options[r3][c].remove(option)
    
    #For each box: NEED TO FINISH
    for r in range(0,9):
        for c in range(0,9):
            
            #List of squares that have equal options, and a count of how many squares have equal options:
            #(Resets variables for the next loop)
            equal_list = []
            equal_count = 1
            
            #Box 1
            if r < 3 and c < 3:                
                startx = 0
                endx = 3
                starty = 0
                endy = 3                            

            #Box 2
            elif r < 3 and 2 < c < 6:                
                startx = 0
                endx = 3
                starty = 3
                endy = 6

            #Box 3
            elif r < 3 and c > 5:                
                startx = 0
                endx = 3
                starty = 6
                endy = 9

            #Box 4
            elif 2 < r < 6 and c < 3:
                startx = 3
                endx = 6
                starty = 0
                endy = 3

            #Box 5
            elif 2 < r < 6 and 2 < c < 6:
                startx = 3
                endx = 6
                starty = 3
                endy = 6

            #Box 6
            elif 2 < r < 6 and c > 5:
                startx = 3
                endx = 6
                starty = 6
                endy = 9

            #Box 7
            elif r > 5 and c < 3:
                startx = 6
                endx = 9
                starty = 0
                endy = 3

            #Box 8
            elif r > 5 and 2 < c < 6:
                startx = 6
                endx = 9
                starty = 3
                endy = 6

            #Box 9
            elif r > 5 and c > 5:
                startx = 6
                endx = 9
                starty = 6
                endy = 9
                
            else:
                print("Error")
                    
            #Check if square's options are equal to any other square in that box:
            for x in range(startx,endx):
                for y in range(starty,endy):
                    if not(x == r and y == c) and options[r][c] == options[x][y]:

                        #Add [r,c] to the list if it is the first time an equal has been found:
                        if equal_count == 1:
                            equal_list.append([r,c])

                        #Add [x,y] if equal and add to the count:
                        equal_list.append([x,y])
                        equal_count += 1

            #Equal_list is complete for square [r,c]
            #If an equal was found, check that the number of equals found is equal to the number of options for [r,c]
            if equal_count == len(options[r][c]) and equal_count > 1:

                #A naked group has been found, remove every option in options[r][c] from every square in the box:
                for option in options[r][c]:
                    for x2 in range(startx,endx):
                        for y2 in range(starty,endy):

                            #Checks if the square checked is not in the naked group:
                            in_group = False
                            for ng_square in equal_list:
                                if x2 == ng_square[0] and y2 == ng_square[1]:
                                    in_group = True

                            #If the square is not in the group, remove the options:
                            if in_group == False:  
                                if option in options[x2][y2]:
                                    options[x2][y2].remove(option)
            
            

#_____________________________________________________________________________________________________________________

file_path = input("Please enter the file path of the Sudoku puzzle you wish to solve:\n")

#Opens the file in read mode:
puzzle_file = open(file_path, "r")

#Reads the file and splits into a list of rows:
puzzle = puzzle_file.read()
puzzle = puzzle.splitlines()

#Turns every row in the list into another list of ints. Format: puzzle[row][col] = value
for x in range(len(puzzle)):
    puzzle[x] = list(puzzle[x])
    
    for y in range(len(puzzle[x])):
        puzzle[x][y] = int(puzzle[x][y])

        
#Creates a list of options for every square in the puzzle. Format: options[row][col] = list of options
#IMPORTANT NOTE: Careful when making lists equal to other lists, copies are not made and changes apply to both!
#Even when I made it equal to a copy of the list, things were going wrong.
options = [[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]]

for x in range(len(options)):
    
    for y in range(len(options[x])):
        options[x][y] = GenerateOptions(x,y) #BUG HERE
        

#Applies OnePossible() and OnlyPossible() functions to make moves, until no more moves can be made:
moves_made = 1000

#Applies OnePossible(), OnlyPossible() and NakedGroups() functions to make moves, until no more moves can be made:
while moves_made > 0:
    moves_made = 0
    moves_made1 = OnePossible()
    moves_made2 = OnlyPossible()
    NakedGroups()
    moves_made = moves_made1 + moves_made2
    
#Prints solved puzzle:
print()
print("The solved puzzle is:")

for x in puzzle:
    print(x)

#Closes the file:
puzzle_file.close()

Please enter the file path of the Sudoku puzzle you wish to solve:
C:\Users\Ariel\Python Workspace\Sudoku1.txt

The solved puzzle is:
[4, 8, 3, 9, 2, 1, 6, 5, 7]
[9, 6, 7, 3, 4, 5, 8, 2, 1]
[2, 5, 1, 8, 7, 6, 4, 9, 3]
[5, 4, 8, 1, 3, 2, 9, 7, 6]
[7, 2, 9, 5, 6, 4, 1, 3, 8]
[1, 3, 6, 7, 9, 8, 2, 4, 5]
[3, 7, 2, 6, 8, 9, 5, 1, 4]
[8, 1, 4, 2, 5, 3, 7, 6, 9]
[6, 9, 5, 4, 1, 7, 3, 8, 2]
